# Download NLP question answering model from NM sparsezoo

This first notebook is used to connect to sparsezoo from NeuralMagic and download the BERT NLP sparsified models for Question Answering: https://sparsezoo.neuralmagic.com/?domain=nlp&sub_domain=question_answering&page=1 from HuggingFace/NeuralMagic: https://github.com/neuralmagic/deepsparse/tree/main/examples/huggingface-transformers#deepsparse-pipeline-example.
For all the models (not fine tuned with new data) we test the Q/A using one example from OS-climate.

## Import libraries

In [55]:
from sparsezoo import Zoo
from transformers import BertTokenizer
from deepsparse import compile_model

from transformers import BertForQuestionAnswering

import torch
import pandas as pd

pd.set_option('max_colwidth', 700)

In [33]:
path_to_model_repo = "/opt/app-root/src/aicoe-osc-demo/models/nm-model/"

## Identify pre-trained models for NLP question answering from Neural Magic

In [3]:
models = Zoo.search_models(domain="nlp", sub_domain="question_answering")

for model in models:
    print(model.stub)

nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_6layers-aggressive_96
nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_quant_3layers-aggressive_84
nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_3layers-aggressive_86
nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned-conservative
nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned-aggressive_98
nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_3layers-aggressive_89
nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_3layers-aggressive_90
nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_quant_6layers-aggressive_91
nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_quant_3layers-aggressive_89
nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_6layers-aggressive_98
nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_quant-moderate
nlp/question_answering/bert-base/pytor

## Inference with NM sparsed BERT models from Hugging Face

In [4]:
context = """the paris agreement on climate change drafted in 2015 aims to reduce worldwide emissions of greenhouse
gases to a level intended to limit a rise in global temperatures to below 2 degrees or, better still,
to below 1.5 degrees. verbund’s target of reducing greenhouse gas emissions by 90% measured beginning from 
the basis year 2011 5 million tonnes co2e until 2021 includes scope 1, scope 2 market- based and parts of scope 3 emissions 
for energy and air travel. the science based targets initiative validated this goal as science-based in october 2016, 
i.e. it meets global standards. according to current planning, the target can be achieved. 
however, if the grid operator requires higher generation volumes 
"""
question = "What is the target year for climate commitment?"

In [74]:
from pipelines import pipeline
num_cores=None  # uses all available CPU cores by default
max_length=256

results = []

for model in models:
    print(f"zoo:{model.stub}")
    qa_pipeline = pipeline(
        task="question-answering",
        model_path=f"zoo:{model.stub}",
        num_cores=num_cores,
        max_length=max_length,
    )

    climate_ = qa_pipeline(
    question=question,
    context=context,
    )

    climate_['answer_real'] = "2021"
    climate_['sparse_zoo_stub'] = f"zoo:{model.stub}"
    climate_['model_name'] = model.stub

    results.append(climate_)

zoo:nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_6layers-aggressive_96
zoo:nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_quant_3layers-aggressive_84
zoo:nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_3layers-aggressive_86
zoo:nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned-conservative
zoo:nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned-aggressive_98
zoo:nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_3layers-aggressive_89
zoo:nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_3layers-aggressive_90
zoo:nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_quant_6layers-aggressive_91
zoo:nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_quant_3layers-aggressive_89
zoo:nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_6layers-aggressive_98
zoo:nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_quant-mod

In [75]:
pd.DataFrame(results)

,score,start,end,answer,answer_real,sparse_zoo_stub,model_name
0,0.253756,49,53,2015,2021,zoo:nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_6layers-aggressive_96,nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_6layers-aggressive_96
1,0.244921,49,53,2015,2021,zoo:nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_quant_3layers-aggressive_84,nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_quant_3layers-aggressive_84
2,0.213660,328,332,2011,2021,zoo:nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_3layers-aggressive_86,nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_3layers-aggressive_86
3,0.268172,550,554,2016,2021,zoo:nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned-conservative,nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned-conservative
4,0.093355,328,365,2011 5 million tonnes co2e until 2021,2021,zoo:nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned-aggressive_98,nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned-aggressive_98
5,0.067210,328,354,2011 5 million tonnes co2e,2021,zoo:nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_3layers-aggressive_89,nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_3layers-aggressive_89
6,0.108203,328,332,2011,2021,zoo:nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_3layers-aggressive_90,nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_3layers-aggressive_90
7,0.581250,49,53,2015,2021,zoo:nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_quant_6layers-aggressive_91,nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_quant_6layers-aggressive_91
8,0.335548,328,332,2011,2021,zoo:nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_quant_3layers-aggressive_89,nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_quant_3layers-aggressive_89
9,0.259208,550,554,2016,2021,zoo:nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_6layers-aggressive_98,nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_6layers-aggressive_98


## Download pre-trained model

In [34]:
# SparseZoo stub to pre-trained sparse-quantized ResNet-50 for imagenet dataset
zoo_stub_path = (
    "zoo:nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_quant-moderate"
)

In [35]:
# model = Zoo.download_model_from_stub(
#     zoo_stub_path,
#     override_parent_path=path_to_model_repo
# )

model = Zoo.load_model_from_stub(
    zoo_stub_path,
    override_parent_path=path_to_model_repo
)

In [36]:
model.display_name

'BERT Pruned-Quantized'

In [37]:
model.display_description

'INT8 quantized BERT base uncased with pruned encoder units on SQuAD'

## Download recipe for sparseML for pre-trained model

In [38]:
# recipe_path = Zoo.download_recipe_from_stub(
#     zoo_stub_path,
#     override_parent_path=path_to_model_repo
# )
# print(f"Recipe downloaded to: {recipe_path}")

recipe_path = Zoo.load_recipe_from_stub(
    zoo_stub_path,
    override_parent_path=path_to_model_repo
)

## Inference with BERT model from Hugging Face

In [68]:
bert_model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
bert_tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [76]:
def question_answer(question, text):
    
    #tokenize question and text as a pair
    input_ids = bert_tokenizer.encode(question, text)
    
    #string version of tokenized ids
    tokens = bert_tokenizer.convert_ids_to_tokens(input_ids)
    
    #segment IDs
    #first occurence of [SEP] token
    sep_idx = input_ids.index(bert_tokenizer.sep_token_id)
    #number of tokens in segment A (question)
    num_seg_a = sep_idx+1
    #number of tokens in segment B (text)
    num_seg_b = len(input_ids) - num_seg_a
    
    #list of 0s and 1s for segment embeddings
    segment_ids = [0]*num_seg_a + [1]*num_seg_b
    assert len(segment_ids) == len(input_ids)
    
    #model output using input_ids and segment_ids
    output = bert_model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))

    #reconstructing the answer
    answer_start = torch.argmax(output.start_logits)
    answer_end = torch.argmax(output.end_logits)
    if answer_end >= answer_start:
        answer = tokens[answer_start]
        for i in range(answer_start+1, answer_end+1):
            if tokens[i][0:2] == "##":
                answer += tokens[i][2:]
            else:
                answer += " " + tokens[i]
                
        if answer.startswith("[CLS]"):
            answer = "Unable to find the answer to your question."
    
        print("\nPredicted answer:\n{}".format(answer.capitalize()))
        return answer.capitalize()
    else:
        answer = "Unable to find the answer to your question."
        print("\nPredicted answer:\n{}".format(answer.capitalize()))
        return answer

In [77]:
answer = question_answer(question, context)
#original answer from the dataset
print("Original answer:\n", '2021')


Predicted answer:
2021
Original answer:
 2021


In [78]:
bert_ = {'score': None, 'start': None, 'end': None, 'answer': answer}
bert_['answer_real'] = "2021"
bert_['sparse_zoo_stub'] = None
bert_['model_name'] = 'BertForQuestionAnswering from HuggingFace/transformers'

In [79]:
results.append(bert_)

In [80]:
pd.DataFrame(results)

,score,start,end,answer,answer_real,sparse_zoo_stub,model_name
0,0.253756,49.0,53.0,2015,2021,zoo:nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_6layers-aggressive_96,nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_6layers-aggressive_96
1,0.244921,49.0,53.0,2015,2021,zoo:nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_quant_3layers-aggressive_84,nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_quant_3layers-aggressive_84
2,0.213660,328.0,332.0,2011,2021,zoo:nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_3layers-aggressive_86,nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_3layers-aggressive_86
3,0.268172,550.0,554.0,2016,2021,zoo:nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned-conservative,nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned-conservative
4,0.093355,328.0,365.0,2011 5 million tonnes co2e until 2021,2021,zoo:nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned-aggressive_98,nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned-aggressive_98
5,0.067210,328.0,354.0,2011 5 million tonnes co2e,2021,zoo:nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_3layers-aggressive_89,nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_3layers-aggressive_89
6,0.108203,328.0,332.0,2011,2021,zoo:nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_3layers-aggressive_90,nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_3layers-aggressive_90
7,0.581250,49.0,53.0,2015,2021,zoo:nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_quant_6layers-aggressive_91,nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_quant_6layers-aggressive_91
8,0.335548,328.0,332.0,2011,2021,zoo:nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_quant_3layers-aggressive_89,nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_quant_3layers-aggressive_89
9,0.259208,550.0,554.0,2016,2021,zoo:nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_6layers-aggressive_98,nlp/question_answering/bert-base/pytorch/huggingface/squad/pruned_6layers-aggressive_98
